In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from keras import backend as K
import cv2
import matplotlib.pyplot as plt
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import SGD
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dir="/kaggle/input/signature-verification-dataset/sign_data/train"
train_csv="/kaggle/input/signature-verification-dataset/sign_data/train_data.csv"
test_csv="/kaggle/input/signature-verification-dataset/sign_data/test_data.csv"
test_dir="/kaggle/input/signature-verification-dataset/sign_data/test"

In [ ]:
df_train=pd.read_csv(train_csv)
df_train.sample(10)

In [ ]:
def dataset_train(train_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(0,2000):
        image1_path=os.path.join(train_dir,train_csvfile.iat[i,0])
        image2_path=os.path.join(train_dir,train_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(100,100))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(100,100))
        x2.append(img2)
        y_train.append(train_csvfile.iat[i,2])
#         print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [ ]:
def dataset_test(test_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(2000,4000):
        image1_path=os.path.join(test_dir,test_csvfile.iat[i,0])
        image2_path=os.path.join(test_dir,test_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(100,100))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(100,100))
        x2.append(img2)
        y_train.append(test_csvfile.iat[i,2])
#         print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [ ]:
df_test=pd.read_csv(test_csv)
xt1,xt2,yt=dataset_test(df_test)
xs1,xs2,ys=dataset_train(df_train)

In [ ]:
c=np.array([[1,2],[1,3]])
c

In [ ]:
x1=np.concatenate((xs1,xt1),axis=0)
x2=np.concatenate((xs2,xt2),axis=0)
ytest=np.concatenate((ys,yt),axis=0)
xtest=np.array([x1,x2])

In [ ]:
x1=x1.reshape(x1.shape[0],1,100,100)
x2=x1.reshape(x2.shape[0],1,100,100)

In [ ]:
xtest=np.concatenate((x1,x2),axis=1)

In [ ]:
xtest[1][0]

In [ ]:
plt.imshow(xtest[1][0])

In [ ]:
xtest[:,0].shape

In [ ]:
for train_index, test_index in skf.split(xtest,ytest):
...    print("TRAIN:", train_index, "TEST:", test_index)
...    X_train, X_test = X[train_index], X[test_index]
...    y_train, y_test = y[train_index], y[test_index]

In [ ]:
kfold=KFold(n_splits=10,shuffle=True)
fold_no=1

In [ ]:
skf=StratifiedKFold(n_splits=4,shuffle=True)
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train,test in skf.split(xtest,ytest):
    def dist3(xy):
        x,y=xy
        return K.sqrt(K.mean(K.square(x-y),axis=1,keepdims=True))
# def dist4(xy):
#     x,y=xy
#     ss=K.sum(K.square(x-y),axis=1,keepdims=True)
#     return K.sqrt(ss)/x.shape[1]


    input1=keras.layers.Input(shape=(100,100,1))
    # x=keras.layers.Flatten()(input1)
    x=keras.layers.Conv2D(64,(7,7),activation='relu')(input1)
    x=keras.layers.MaxPooling2D(pool_size=(3,3))(x)
#     x=BatchNormalization()(x)
#     x=keras.layers.Dropout(0.7)(x)
#     x=keras.layers.Conv2D(256,(7,7),activation='relu')(x)
#     x=keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)
#     x=keras.layers.Conv2D(128,(7,7),activation='relu')(x)
#     # x=BatchNormalization()(x)
#     x=keras.layers.MaxPooling2D(2,2)(x)
#     x=keras.layers.Dropout(0.7)(x)
    x=keras.layers.Conv2D(128,(5,5),activation='relu')(x)
    x=keras.layers.MaxPooling2D(pool_size=(3,3),strides=2)(x)
    x=keras.layers.Flatten()(x)
#     x=keras.layers.Dense(4096,activation='relu')(x)
    x=keras.layers.Dense(500,activation='relu')(x)
#     x=BatchNormalization()(x)
    dense=keras.models.Model(inputs=input1,outputs=x)


    img1=keras.layers.Input(shape=(100,100,1))
    img2=keras.layers.Input(shape=(100,100,1))
    dense1=dense(img1)
    dense2=dense(img2)
    fc=keras.layers.Lambda(dist3)([dense1,dense2])
#     fc=keras.layers.Dense(100,activation='relu')(fc)
    fc=keras.layers.Dense(1,activation='sigmoid')(fc)

    m=keras.models.Model(inputs=[img1,img2],outputs=fc)

    m.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    m.summary()
    
    history=m.fit([xtest[:,0][train],xtest[:,1][train]],ytest[train],epochs=10,batch_size=16)
    
    scores=m.evaluate([xtest[:,0][test],xtest[:,1][test]],ytest[test])
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    

In [ ]:
sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
acc_per_fold

In [ ]:

# def dist1(xy):
#     x, y = xy
#     sum_abs = K.sum(K.abs(x - y), axis=1, keepdims=True)
# #     return sum_abs
#     return K.maximum(sum_abs, K.epsilon())
# def dist2(xy):
#     x,y=xy
#     sum_square=K.sum(K.square(x-y),axis=1,keepdims=True)
#     return K.sqrt(K.maximum(sum_square, K.epsilon()))

def dist3(xy):
    x,y=xy
    return K.sqrt(K.mean(K.square(x-y),axis=1,keepdims=True))
# def dist4(xy):
#     x,y=xy
#     ss=K.sum(K.square(x-y),axis=1,keepdims=True)
#     return K.sqrt(ss)/x.shape[1]


input1=keras.layers.Input(shape=(100,100,1))
# x=keras.layers.Flatten()(input1)
x=keras.layers.Conv2D(64,(10,10),activation='relu')(input1)
x=keras.layers.MaxPooling2D(2,2)(x)
# x=BatchNormalization()(x)
x=keras.layers.Dropout(0.5)(x)
# x=keras.layers.Conv2D(128,(7,7),activation='relu')(x)
# x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Conv2D(100,(7,7),activation='relu')(x)
# x=BatchNormalization()(x)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Dropout(0.5)(x)
# x=keras.layers.Conv2D(258,(4,4),activation='relu')(x)
# x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Flatten()(x)
# x=keras.layers.Dense(4096,activation='relu')(x)
x=keras.layers.Dense(500,activation='relu')(x)
# x=BatchNormalization()(x)
dense=keras.models.Model(inputs=input1,outputs=x)


img1=keras.layers.Input(shape=(100,100,1))
img2=keras.layers.Input(shape=(100,100,1))
dense1=dense(img1)
dense2=dense(img2)
fc=keras.layers.Lambda(dist3)([dense1,dense2])
# fc=keras.layers.Dense(100,activation='relu')(fc)
fc=keras.layers.Dense(1,activation='sigmoid')(fc)

m=keras.models.Model(inputs=[img1,img2],outputs=fc)

m.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
m.summary()

    

In [ ]:
m.fit([xs1,xs2],ys,epochs=10,batch_size=16)

In [ ]:
m.evaluate([xt1,xt2],yt)

In [ ]:
def check(stored,image):
    img2=np.array([image])
    for i in range(0,stored.shape[0]):
        img1=np.array([stored[i]])
        pred=m.predict([img1,img2])
        print(pred)
        if(pred<0.5):
            print("matched",i)
            return
    print("unmatched")
    

In [ ]:
database_signatures=xt1[:15]

In [ ]:
check(database_signatures,xt1[13])

In [ ]:
plt.imshow(xs1[223])

In [ ]:
plt.imshow(xt1[223])